In [28]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50  # type: ignore
from tensorflow.keras.models import Sequential  # type: ignore
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization  # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
from tensorflow.keras.optimizers import Adam  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # type: ignore

In [29]:
# Load the ResNet50 model, pre-trained on ImageNet, excluding the top (classification) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [30]:
# Freeze the pre-trained layers so they don't get updated during the first training phase
base_model.trainable = False

In [31]:
# Build the model
model = Sequential([
    base_model,  # Add the pre-trained ResNet base model
    Flatten(),  # Flatten the output of the ResNet
    Dense(1024, activation='relu'),  # Fully connected layer
    Dropout(0.5),  # Add dropout to reduce overfitting
    BatchNormalization(),
    Dense(512, activation='relu'),  # Fully connected layer
    Dropout(0.5),
    BatchNormalization(),
    Dense(5, activation='softmax')  # Output layer for 5 classes
])

In [32]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [33]:
# Data augmentation for training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=30,
                                   brightness_range=[0.8, 1.2],
                                   width_shift_range=0.2,
                                   height_shift_range=0.2)

In [34]:
# No augmentation for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
# Load data
training_set = train_datagen.flow_from_directory(
    r'C:\\Con!\\Terrain-Recognition\\Training Data',
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    r'C:\\Con!\\Terrain-Recognition\\Testing Data',
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical')

Found 10000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [36]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [37]:
# Print number of samples in training and test sets for verification
print(f"Training set samples: {training_set.samples}")
print(f"Test set samples: {test_set.samples}")

Training set samples: 10000
Test set samples: 500


In [38]:
# Calculate steps_per_epoch and validation_steps based on dataset size
train_steps = training_set.samples // training_set.batch_size
val_steps = test_set.samples // test_set.batch_size

In [39]:

# Define callbacks for learning rate adjustment and early stopping
lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [40]:
# Train the model
history = model.fit(
    training_set,
    steps_per_epoch=train_steps,
    epochs=20,
    validation_data=test_set,
    validation_steps=val_steps,
    callbacks=[lr_reduction, early_stopping]
)

Epoch 1/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 154s 912ms/step - accuracy: 0.2337 - loss: 2.0772 - val_accuracy: 0.3661 - val_loss: 1.4757 - learning_rate: 0.0010
Epoch 2/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2031 - loss: 1.8549 - val_accuracy: 0.3654 - val_loss: 1.4631 - learning_rate: 0.0010
Epoch 3/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 56s 354ms/step - accuracy: 0.2864 - loss: 1.7190 - val_accuracy: 0.2031 - val_loss: 2.0191 - learning_rate: 0.0010
Epoch 4/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2344 - loss: 1.7610 - val_accuracy: 0.1923 - val_loss: 2.0419 - learning_rate: 0.0010
Epoch 5/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 62s 393ms/step - accuracy: 0.2918 - loss: 1.6090 - val_accuracy: 0.2121 - val_loss: 2.8729 - learning_rate: 0.0010
Epoch 6/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3125 - loss: 1.5465 - val_accuracy: 0.0962 - val_loss: 3.1937 - learning_rate: 5.0000e-04
Epoch 7/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 63s 399ms/step - accuracy: 0.3201 

In [41]:

# Save the model
model.save('resnet50_terrain_model.keras')